 ### 아래 내용은 글로벌 기업 (주)인텔에서 개발한 <br> Intel® AI for Youth Program 내용을 
대한민국 Implementation Partner사인 (주)Brain AI와 Brain AI Lead Coach Network에서 우리나라 초, 중, 고 학생들의 AI 교육을 위해 현지화 한 내용입니다. <br>
(주)Brain AI와 NDA를 체결한 학교에서만 사용 가능하며 NDA를 준수해야합니다. <br>
상업적 사용은 불가하며, 학교내에서 학생들 교육활동에 자유롭게 사용가능합니다.

# 프로젝트 제목: Brain AI_Multisorter
구글 티처블 머신에서 개발한 인공지능 모델을 불러와 마이크로비트와 마이크로비트 확장보드, 서보모터로 작동하는 인공지능 자동 분류기 개발하기

In [1]:
# 필요 라이브러리 불러오기
import numpy as np
import cv2
import time

import serial
import serial.tools.list_ports

import tensorflow.keras
from tensorflow.keras.models import model_from_json

In [2]:
# USB 시리얼 포트 자동 찾기
ports = serial.tools.list_ports.comports()
com = ''
for port, desc, hwid in sorted(ports):        
    if 'USB' in desc:
        com = port
if com != '':
    print('Micro:bit detected: ', com)   
else:
    print('Please connect your microbit to this PC via USB')     

Micro:bit detected:  COM12


In [3]:
# 레이블 불러오기
labels = []
file = open("model/labels.txt", "r")
for x in file:
    labels.append(x.rstrip('\n'))
print('labels = ', labels)
file.close()

labels =  ['0 None', '1 Red', '2 Orange', '3 Yellow', '4 Green', '5 Blue', '6 Brown']


In [4]:
#인공지능 모델 불러오기
model = tensorflow.keras.models.load_model('model/keras_model.h5', compile = False)

In [5]:
# 시리얼 통신으로 컴퓨터에 연결되어 있는 마이크로비트에 명령 보내기
def SerialSendCommand(cmd):
    cmd = str(cmd) 
    cmd  = cmd + '\n'
    cmd = str.encode(cmd) 
    ser.write(cmd)

In [6]:
# 새로운 이미지 데이터 읽어와 분류 예측하기
def ReadPicture(cropped, model):   
    
    input_width = 224
    input_height = 224

    resized_image = cv2.resize(cropped, (input_width, input_height))
    imgRGB =cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)
    
    normalized_image = (imgRGB.astype(np.float32) / 127.0) - 1
    batch = normalized_image.reshape(1, input_height, input_width, 3)

    prediction = model.predict(batch)
    
    class_id = np.argmax(prediction)
    score = np.max(prediction) 
    
    return class_id, score

### Teachable Machine Guide 
- Create the array of the right shape to feed into the keras model
- The 'length' or number of images you can put into the array is
- determined by the first position in the shape tuple, in this case 1.
- data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

In [7]:
# 메인 함수
def Main():
        
    MODE_PAUSE = -1 
    MODE_START = 0
    mode_status = MODE_PAUSE
 
    tic = time.time()
    text = "Press 'p' to start"
    color  = (0, 0, 255)
    cmd = 90
    print("Press [q] to quit") 
  
    video_capture = cv2.VideoCapture(1)
    height = video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT )
    width = video_capture.get(cv2.CAP_PROP_FRAME_WIDTH )

    y1 = int(height - (height * top))
    y2 = int(height - (height * bottom))
    square = (y2-y1)/2
    x1 = int(width/2 - square)
    x2 = int(width/2 + square)
    
    while(True):

        grabbed, frame = video_capture.read()
        cropped = frame[y1:y2, x1:x2]
 
        if mode_status == MODE_START:
        
            if time.time() - tic > 2.5:      
                tic = time.time() 
                
                class_id, score = ReadPicture(cropped, model)   
                text = labels[class_id][1:] + ' ' + str(int(score*100)) + '%' 
                if score >= .6:
                    cmd = labels[class_id][0]
                    SerialSendCommand(cmd)
                    
        cv2.putText(frame, text, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        
        cv2.imshow("Frame", frame) 
        
        key = cv2.waitKey(1)
        
        if key == ord("q"):
            break

        elif key == ord('p'):
            if mode_status != MODE_PAUSE:
                mode_status = MODE_PAUSE       
                color = (0, 0, 255)
                text = "Press 'p' to start"
                SerialSendCommand("pause")

            elif mode_status == MODE_PAUSE:
                mode_status = MODE_START
                color = (0, 255, 0)
                text = "Press 'p' to pause"
                
    ser.close()
    video_capture.release()
    cv2.destroyAllWindows()

In [9]:
#설정 값 변경 및 프로그램 실행
ser = serial.Serial(com, 9600, timeout=0,parity=serial.PARITY_NONE, rtscts=0)  
top = .65
bottom = .25

if __name__ == '__main__': 
    Main()

Press [q] to quit
